In [2]:
import numpy as np
import cv2
from collections import Counter

In [3]:
def haar_transform(image):
    rows, cols = image.shape
    row_transform = np.zeros_like(image, dtype=np.float32)
    for i in range(rows):
        for j in range(0, cols - 1, 2):
            row_transform[i, j // 2] = (image[i, j] + image[i, j + 1]) / 2
            row_transform[i, (j // 2) + cols // 2] = (image[i, j] - image[i, j + 1]) / 2
    result = np.zeros_like(row_transform, dtype=np.float32)
    for j in range(cols):
        for i in range(0, rows - 1, 2):
            result[i // 2, j] = (row_transform[i, j] + row_transform[i + 1, j]) / 2
            result[(i // 2) + rows // 2, j] = (row_transform[i, j] - row_transform[i + 1, j]) / 2
    LL = result[:rows // 2, :cols // 2]  
    LH = result[:rows // 2, cols // 2:]  
    HL = result[rows // 2:, :cols // 2] 
    HH = result[rows // 2:, cols // 2:]

    return LL, LH, HL, HH

In [4]:

def quantize(coeffs, n_quants):
    min_val = np.min(coeffs)
    max_val = np.max(coeffs)
    step = (max_val - min_val) / n_quants
    quantized = np.round((coeffs - min_val) / step).astype(int)
    return quantized, min_val, step

In [5]:
def run_length_encode(data):
    encoded = []
    for value, count in Counter(data.flatten()).items():
        encoded.append((value, count))
    return encoded

In [6]:
img_path = 'pic.png'
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

img = img.astype(np.float32)
np.savetxt('image.txt', img, fmt='%d')

In [7]:
LL, LH, HL, HH = haar_transform(img)
n_quants = 4

LH_q, LH_min, LH_step = quantize(LH, n_quants)
HL_q, HL_min, HL_step = quantize(HL, n_quants)
HH_q, HH_min, HH_step = quantize(HH, n_quants)
LH_rle = run_length_encode(LH_q)
HL_rle = run_length_encode(HL_q)
HH_rle = run_length_encode(HH_q)

In [8]:
with open('wavelet_data.txt', 'w') as f:
    np.savetxt(f, LL, fmt='%d')
    f.write('\n')
    for value, count in LH_rle:
        f.write(f"{value} {count}\n")
    f.write('\n')
    for value, count in HL_rle:
        f.write(f"{value} {count}\n")
    f.write('\n')
    for value, count in HH_rle:
        f.write(f"{value} {count}\n")

original_size = img.nbytes
compressed_size = 0
with open('wavelet_data.txt', 'r') as f:
    compressed_size = len(f.read().encode('utf-8'))

print(f"Размер исходного изображения: {original_size} байт")
print(f"Размер сжатого изображения: {compressed_size} байт")
print(f"Степень сжатия: {original_size / compressed_size:.2f}")

Размер исходного изображения: 8538048 байт
Размер сжатого изображения: 1448007 байт
Степень сжатия: 5.90
